# DDA (Deep Discriminant Analysis)

기존의 DDA 학습은 classification base였다면 이번에는 Metric learning을 이용하여 DDA를 수행해보자.

----
아직은 결과가 그렇게 좋지않다. ㅠㅠ
Hard mining까지 썼는데도 그렇다.

parameter 변경해 볼 것이 있나?

### Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

### Configuration

In [3]:
from sv_system.utils.parser import set_train_config
import easydict

# datasets
# voxc1_fbank_xvector
# gcommand_fbank_xvector

args = easydict.EasyDict(dict(dataset="voxc1_fbank_xvector",
                              input_frames=100, splice_frames=[50, 100], stride_frames=1, input_format='fbank',
                              cuda=True,
                              lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=False,
                              batch_size=128,
                              arch="tdnn_conv", loss="softmax",
                              n_epochs=50
                             ))
config = set_train_config(args)

### Dataset

In [4]:
import torch.utils.data as data

class embedDataset(data.Dataset):
    def __init__(self, embeds, labels):
        super().__init__()
        self.embeds = embeds
        self.labels = labels
        
    def __getitem__(self, index):
        
        return self.embeds[index], self.labels[index]
    
    def __len__(self):
        
        return self.embeds.shape[0]

In [5]:
def embedToDataset(embeds, key_df):
    labels = key_df.label.tolist()
    dataset = embedDataset(embeds, labels)
    
    return dataset, embeds.shape[1], len(key_df.label.unique())

In [6]:
trial = pd.read_pickle("../dataset/dataframes/voxc1/voxc_trial.pkl")

In [7]:
si_keys = pickle.load(open("../embeddings/voxc12/xvectors/xvectors_tdnn7b/train_feat/key.pkl", "rb"))
si_embeds = np.load("../embeddings/voxc12/xvectors/xvectors_tdnn7b/train_feat/feat.npy")

sv_keys = pickle.load(open("../embeddings/voxc12/xvectors/xvectors_tdnn7b/test_feat/key.pkl", "rb"))
sv_embeds = np.load("../embeddings/voxc12/xvectors/xvectors_tdnn7b/test_feat/feat.npy")

In [8]:
# voxc1_keys = embed_keys[embed_keys.origin == 'voxc1']

In [9]:
def key2df(keys):
    key_df = pd.DataFrame(keys, columns=['key'])
    key_df['spk'] = key_df.key.apply(lambda x: x.split("-")[0])
    key_df['label'] = key_df.groupby('spk').ngroup()
    key_df['origin'] = key_df.spk.apply(lambda x: 'voxc2' if x.startswith('id') else 'voxc1')
    
    return key_df

In [10]:
si_key_df = key2df(si_keys)
sv_key_df = key2df(sv_keys)

In [11]:
si_dataset, embed_dim, n_labels = embedToDataset(si_embeds, si_key_df)
sv_dataset, _, _ = embedToDataset(sv_embeds, sv_key_df)

### Batch Sampler

In [12]:
import math
import random
import itertools

def index_dataset(dataset):
    return {c : [example_idx for example_idx, (_, class_label_ind) in \
                 enumerate(zip(dataset.embeds, dataset.labels)) if class_label_ind == c] for c in set(dataset.labels)}

def sample_from_class(images_by_class, class_label_ind):
    return images_by_class[class_label_ind][random.randrange(len(images_by_class[class_label_ind]))]

def simple(batch_size, dataset, prob_other = 0.5):
    '''lazy sampling, not like in lifted_struct. they add to the pool all postiive combinations, then compute the average number of positive pairs per image, then sample for every image the same number of negative pairs'''
    images_by_class = index_dataset(dataset)
    for batch_idx in range(int(math.ceil(len(dataset) * 1.0 / batch_size))):
        example_indices = []
        for i in range(0, batch_size, 2):
            perm = random.sample(images_by_class.keys(), 2)
            example_indices += [sample_from_class(images_by_class, perm[0]), sample_from_class(images_by_class, perm[0 if i == 0 or random.random() > prob_other else 1])]
        yield example_indices[:batch_size]

def triplet(batch_size, dataset, class2img=None):
    if class2img is not None:
        images_by_class = class2img
    else:
        images_by_class = index_dataset(dataset)
    for batch_idx in range(int(math.ceil(len(dataset) * 1.0 / batch_size))):
        example_indices = []
        for i in range(0, batch_size, 3):
            perm = random.sample(images_by_class.keys(), 2)
            example_indices += [sample_from_class(images_by_class, perm[0]), sample_from_class(images_by_class, perm[0]), sample_from_class(images_by_class, perm[1])]
        yield example_indices[:batch_size]

def npairs(batch_size, dataset, K = 4):
    images_by_class = index_dataset(dataset)
    for batch_idx in range(int(math.ceil(len(dataset) * 1.0 / batch_size))):
        example_indices = [sample_from_class(images_by_class, class_label_ind) for k in range(int(math.ceil(batch_size * 1.0 / K))) for class_label_ind in [random.choice(images_by_class.keys())] for i in range(K)]
        yield example_indices[:batch_size]

In [13]:
# use values to exclude unnecessary index
si_key_df['num_id'] = range(len(si_key_df))
si_class2idx = si_key_df.groupby('label').apply(lambda x: x.num_id.values).to_dict()

### Dataloader

In [14]:
adapt_sampler = lambda batch, dataset, sampler, **kwargs: \
type('', (torch.utils.data.sampler.Sampler,), 
     dict(__len__ = dataset.__len__, __iter__ = \
          lambda _: itertools.chain.from_iterable(sampler(batch, dataset, **kwargs))))(dataset)

In [15]:
import torch
from torch.utils.data.dataloader import DataLoader

# si_loader = DataLoader(si_dataset, batch_size=128, num_workers=4, shuffle=True)
n_pairs_per_batch = 23
batch_size = n_pairs_per_batch * 3

si_loader = torch.utils.data.DataLoader(si_dataset, 
                                       sampler = adapt_sampler(batch_size, si_dataset, triplet, class2img=si_class2idx), 
                                       num_workers = 8, batch_size = batch_size, 
                                       drop_last = True, pin_memory = True)
sv_loader = DataLoader(sv_dataset, batch_size=128, num_workers=4, shuffle=False)

### Model Define

In [16]:
import torch.nn as nn

class dda_model(nn.Module):
    def __init__(self, in_dims, n_labels):
        super().__init__()
        
        self.input_layer = nn.Sequential(
            nn.Linear(in_dims, 2*in_dims),
            nn.PReLU()
        )
        
        self.hidden_layer = nn.Sequential(
            nn.Linear(2*in_dims, 2*in_dims),
            nn.PReLU()
        )    
        self.hidden_batch = nn.BatchNorm1d(2*in_dims)
    
#         self.embedding_layer = nn.Linear(2*in_dims, n_labels)
        
    def embed(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.hidden_batch(x)
    
        return x
    
    def forward(self, x):           
        x = self.embed(x)
        
        return x

### Model Train

In [17]:
import torch.nn.functional as F

def embeds_utterance(config, val_dataloader, model):
#     val_iter = iter(val_dataloader)
    embeddings = []
    labels = []
    model.eval()

    with torch.no_grad():
        for batch in val_dataloader:
            X, y = batch
            if not config['no_cuda']:
                X = X.cuda()
                
            model_output = model.embed(X).cpu().detach()
            embeddings.append(model_output)
            labels.append(y.numpy())
        embeddings = torch.cat(embeddings)
        labels = np.hstack(labels)
    return embeddings, labels 

def sv_test(config, sv_loader, model, trial):
        embeddings, _ = embeds_utterance(config, sv_loader, model)
        sim_matrix = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
        cord = [trial.enrolment_id.tolist(), trial.test_id.tolist()]
        score_vector = sim_matrix[cord].numpy()
        label_vector = np.array(trial.label)
        fpr, tpr, thres = roc_curve(
                label_vector, score_vector, pos_label=1)
        eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

        return eer, label_vector, score_vector
    
def sv_euc_test(config, sv_loader, model, trial):
        embeddings, _ = embeds_utterance(config, sv_loader, model)
#         embeddings /= embeddings.norm(dim=1,keepdim=True)
        a = embeddings.unsqueeze(1)
        b = embeddings.unsqueeze(0)
        dist = a - b
#         sim_matrix = -dist.norm(dim=2)
        sim_matrix = -dist.pow(2).sum(2).sqrt()
        cord = [trial.enrolment_id.tolist(), trial.test_id.tolist()]
        score_vector = sim_matrix[cord].numpy()
        label_vector = np.array(trial.label)
        fpr, tpr, thres = roc_curve(
                label_vector, score_vector, pos_label=1)
        eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

        return eer, label_vector, score_vector

In [18]:
def hard_mining(anchor, pos_egs, neg_egs, margin=1.0):
    pos_dist = (anchor - pos_egs).pow(2).sum(1)
    pos_dist = torch.clamp(pos_dist, min=0)
#     pos_dist = pos_dist.sqrt()
    
    neg_dist = (anchor - neg_egs).pow(2).sum(1)
    neg_dist = torch.clamp(neg_dist, min=0)
#     neg_dist = neg_dist.sqrt()
    
    
    hard_pos_dist = pos_dist.max()
    hard_neg_dist = neg_dist.min()
    
#     print(f"hard_pos:{hard_pos_dist}, hard_neg:{hard_neg_dist}")
    
    triplet_loss = torch.clamp(hard_pos_dist - hard_neg_dist + margin, min=0)
    triplet_loss = torch.sum(triplet_loss)
    
    return triplet_loss

In [19]:
import torch

def triplet_train(model, loader, criterion , margin):
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(loader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()

        embeds = model(X)
#         embeds = embeds / embeds.norm(dim=1,keepdim=True)
        anchor = embeds[0:batch_size:3]
        pos_egs = embeds[1:batch_size:3]
        neg_egs = embeds[2:batch_size:3]
        loss = criterion(anchor, pos_egs, neg_egs, margin=margin)
        loss.backward()
        optimizer.step()
                        
        loss_sum += loss.item()
        total += y.size(0)
        if (batch_idx+1) % 1000 == 0:
            print("Batch {}/{}\t Loss {:.6f}" \
                  .format(batch_idx+1, len(loader), loss_sum / total))
    return loss_sum / total

In [20]:
model = dda_model(embed_dim, n_labels) 

In [21]:
if not config['no_cuda']:
    model = model.cuda()

In [22]:
from sv_system.train.train_utils import set_seed, find_optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR

config['lrs'] = [0.1, 0.01]
config
_, optimizer = find_optimizer(config, model)
semi_hard_mining = nn.TripletMarginLoss(margin=1.0, p=2, size_average=False)
plateau_scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)
step_scheduler = MultiStepLR(optimizer, [30], 0.1)

In [ ]:
from sv_system.train.si_train import val
from sklearn.metrics import roc_curve
from tensorboardX import SummaryWriter

writer = SummaryWriter("logs/xvector_eucl_hard_m0.4.tf.log")

for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))

#     step_scheduler.step()    
    
#     train code
    train_loss = triplet_train(model, si_loader, hard_mining, margin=0.4)
    print("epoch #{}, train loss: {}".format(epoch_idx, train_loss))
    writer.add_scalar("train/loss", train_loss, epoch_idx+1)

#     evaluate best_metric
    if not config['no_eer']:
        # eer validation code
        eer, label, score = sv_euc_test(config, sv_loader, model, trial)
        print("epoch #{}, sv eer: {}".format(epoch_idx, eer))
        writer.add_scalar("sv_test/eer", eer, epoch_idx+1)
    
    plateau_scheduler.step(train_loss)

------------------------------
curr_lr: 0.1
Batch 1000/18512	 Loss 0.050067
Batch 2000/18512	 Loss 0.040712
Batch 3000/18512	 Loss 0.037593
Batch 4000/18512	 Loss 0.035952
Batch 5000/18512	 Loss 0.035083
Batch 6000/18512	 Loss 0.035693
Batch 7000/18512	 Loss 0.036593
Batch 8000/18512	 Loss 0.036827
Batch 9000/18512	 Loss 0.036618
Batch 10000/18512	 Loss 0.036537
Batch 11000/18512	 Loss 0.036291
Batch 12000/18512	 Loss 0.036363
Batch 13000/18512	 Loss 0.036110
Batch 14000/18512	 Loss 0.036122
Batch 15000/18512	 Loss 0.036119
Batch 16000/18512	 Loss 0.035961
Batch 17000/18512	 Loss 0.035918
Batch 18000/18512	 Loss 0.035806
epoch #0, train loss: 0.03572362130030346
epoch #0, sv eer: 0.1235225215631988
------------------------------
curr_lr: 0.1
Batch 1000/18512	 Loss 0.034303
Batch 2000/18512	 Loss 0.035074
Batch 3000/18512	 Loss 0.035319
Batch 4000/18512	 Loss 0.035163
Batch 5000/18512	 Loss 0.035120
Batch 6000/18512	 Loss 0.035012
Batch 7000/18512	 Loss 0.034760
Batch 8000/18512	 Loss 0

Batch 13000/18512	 Loss 0.032624
Batch 14000/18512	 Loss 0.032639
Batch 15000/18512	 Loss 0.032679
Batch 16000/18512	 Loss 0.032693
Batch 17000/18512	 Loss 0.032687
Batch 18000/18512	 Loss 0.032610
epoch #11, train loss: 0.03258104128504993
epoch #11, sv eer: 0.12314982429986157
------------------------------
curr_lr: 0.1
Batch 1000/18512	 Loss 0.032547
Batch 2000/18512	 Loss 0.032308
Batch 3000/18512	 Loss 0.032323
Batch 4000/18512	 Loss 0.032313
Batch 5000/18512	 Loss 0.032591
Batch 6000/18512	 Loss 0.032594
Batch 7000/18512	 Loss 0.032886
Batch 8000/18512	 Loss 0.032942
Batch 9000/18512	 Loss 0.033061
Batch 10000/18512	 Loss 0.033004
Batch 11000/18512	 Loss 0.032925
Batch 12000/18512	 Loss 0.032930
Batch 13000/18512	 Loss 0.032983
Batch 14000/18512	 Loss 0.032824
Batch 15000/18512	 Loss 0.032870
Batch 16000/18512	 Loss 0.032852
Batch 17000/18512	 Loss 0.032873
Batch 18000/18512	 Loss 0.032827
epoch #12, train loss: 0.03279097461724666
epoch #12, sv eer: 0.1299648599723139
----------

In [ ]:
torch.save(dda_net.state_dict(), open("temp_dda_net.pt", "wb"))

### Original EER

In [ ]:
embeddings = torch.from_numpy(sv_embeds)

sim_matrix = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
# sim_matrix = -(embeddings.unsqueeze(1)-embeddings.unsqueeze(0)).pow(2).sum(2)
cord = [trial.enrolment_id.tolist(), trial.test_id.tolist()]
score_vector = sim_matrix[cord].numpy()
label_vector = np.array(trial.label)
fpr, tpr, thres = roc_curve(
        label_vector, score_vector, pos_label=1)
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

print(f"Original EER: {eer}")

In [ ]:
label_vector

In [ ]:
-score_vector[label_vector].mean()

In [ ]:
-score_vector[1-label_vector].mean()

In [ ]:
score_vector

### Extracting new embeddings

In [ ]:
dda_net.load_state_dict(torch.load("temp_dda_net.pt"))

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=1, shuffle=False)

In [ ]:
new_embeds = []
for (X, y) in test_dataloader:
        if is_cuda:
            X = X.cuda()
        new_embed = dda_net.embed(X)
        new_embeds += [new_embed]

In [ ]:
new_embed_tensor = torch.cat(new_embeds, dim=0)

In [ ]:
import torch.nn.functional as F
sim_matrix = F.cosine_similarity(                                                                                                                                                                                                                                    
     new_embed_tensor.cpu().unsqueeze(1), new_embed_tensor.cpu().unsqueeze(0), dim=2)       

In [ ]:
from sklearn.metrics import roc_curve

cord = [trial.enrolment_id.tolist(), trial.test_id.tolist()]                                                                                                                                                                                                         
score_vector = sim_matrix[cord].detach().numpy()                                                                                                                                                                                                                              
label_vector = np.array(trial.label)                                                                                                                                                                                                                                 
fpr, tpr, thres = roc_curve(                                                                                                                                                                                                                                         
     label_vector, score_vector, pos_label=1)                                                                                                                                                                                                                     
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

In [ ]:
eer